# prueba tecnica 

URL:https://raw.githubusercontent.com/rashida048/Datasets/master/home_data.csv
1. Realice un análisis exploratorio del conjunto de datos y presente sus
conclusiones.
2. Realice un preprocesamiento de los datos y explique el proceso realizado.
3. Seleccione un modelo para ajustar el precio de las viviendas teniendo en
cuenta métricas de validación y testeo, explique cuáles modelos presentó y
el proceso de selección.
4. A partir del modelo seleccionado, y asumiendo que este es un estimador
acertado del valor de mercado, ¿Cómo seleccionaría usted viviendas
atractivas para inversión? De un ejemplo de una.

# Análisis Exploratorio

In [22]:
import pandas as pd

url = "https://raw.githubusercontent.com/rashida048/Datasets/master/home_data.csv"
df = pd.read_csv(url)
print(df.head())
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")

           id             date   price  bedrooms  bathrooms  sqft_living  \
0  7129300520  20141013T000000  221900         3       1.00         1180   
1  6414100192  20141209T000000  538000         3       2.25         2570   
2  5631500400  20150225T000000  180000         2       1.00          770   
3  2487200875  20141209T000000  604000         4       3.00         1960   
4  1954400510  20150218T000000  510000         3       2.00         1680   

   sqft_lot  floors  waterfront  view  ...  grade  sqft_above  sqft_basement  \
0      5650     1.0           0     0  ...      7        1180              0   
1      7242     2.0           0     0  ...      7        2170            400   
2     10000     1.0           0     0  ...      6         770              0   
3      5000     1.0           0     0  ...      7        1050            910   
4      8080     1.0           0     0  ...      8        1680              0   

   yr_built  yr_renovated  zipcode      lat     long  sqft_liv

## caracteristicas de las viviendas y tipos de datos
- id: identificador único de la vivienda (entero)
- date: fecha de la venta (cadena/fecha)
- price: precio de la vivienda (float)
- bedrooms: número de habitaciones (entero)
- bathrooms: número de baños (entero/categórico)
- sqft_living: área habitable en pies cuadrados (entero)
- sqft_lot: área total del lote (entero)
- floors: número de pisos (entero/categórico)
- waterfront: si tiene vista al mar (1: sí, 0: no) (entero/categórico)
- view: calidad de la vista (entero/categórico)
- condition: condición de la vivienda (entero/categórico)
- grade: calificación de la construcción (entero/categórico)
- sqft_above: área sobre el suelo (entero)
- sqft_basement: área del sótano (entero)
- yr_built: año de construcción (entero)
- yr_renovated: año de renovación (entero)
- zipcode: código postal (cadena/categórico)
- lat: latitud (float)
- long: longitud (float)
- sqft_living15: área habitable de las 15 casas más cercanas (entero)
- sqft_lot15: área de lote de las 15 casas más cercanas (entero)


In [18]:
# Revisar valores nulos y duplicados en el DataFrame
print(df.isnull().sum())
print("numero de duplicados:",df.duplicated().sum())

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64
numero de duplicados: 0


In [23]:
# Extraer año y mes de la columna 'date'
df['year_sold'] = df['date'].str[:4].astype(int)
df['month_sold'] = df['date'].str[4:6].astype(int)

# Calcular años desde la renovación (si no fue renovada, usar año de construcción)
df['last_renovation'] = df[['yr_built', 'yr_renovated']].max(axis=1)
df['years_since_renovation'] = df['year_sold'] - df['last_renovation']

print(df[['date', 'year_sold', 'month_sold', 'yr_built', 'yr_renovated', 'years_since_renovation']].head())

              date  year_sold  month_sold  yr_built  yr_renovated  \
0  20141013T000000       2014          10      1955             0   
1  20141209T000000       2014          12      1951          1991   
2  20150225T000000       2015           2      1933             0   
3  20141209T000000       2014          12      1965             0   
4  20150218T000000       2015           2      1987             0   

   years_since_renovation  
0                      59  
1                      23  
2                      82  
3                      49  
4                      28  


In [24]:
# Separar columnas categóricas y no categóricas según el significado de los datos
categoricas = [
    'floors','waterfront', 'view', 'condition', 'grade', 'zipcode','year_sold','month_sold'
 ]
no_categoricas = [
     'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement',
    'yr_built', 'lat', 'long', 'sqft_living15', 'sqft_lot15' ,'years_since_renovation'
 ]

# DataFrame de variables categóricas
df_categoricas = df[categoricas]
print('Variables categóricas:')
for col in df_categoricas.columns:
    print(f'\n{col} value counts:')
    print(df_categoricas[col].value_counts())

# DataFrame de variables no categóricas
df_no_categoricas = df[no_categoricas]
print('\nVariables no categóricas:')
print(df_no_categoricas.describe())

Variables categóricas:

floors value counts:
floors
1.0    10680
2.0     8241
1.5     1910
3.0      613
2.5      161
3.5        8
Name: count, dtype: int64

waterfront value counts:
waterfront
0    21450
1      163
Name: count, dtype: int64

view value counts:
view
0    19489
2      963
3      510
1      332
4      319
Name: count, dtype: int64

condition value counts:
condition
3    14031
4     5679
5     1701
2      172
1       30
Name: count, dtype: int64

grade value counts:
grade
7     8981
8     6068
9     2615
6     2038
10    1134
11     399
5      242
12      90
4       29
13      13
3        3
1        1
Name: count, dtype: int64

zipcode value counts:
zipcode
98103    602
98038    590
98115    583
98052    574
98117    553
        ... 
98102    105
98010    100
98024     81
98148     57
98039     50
Name: count, Length: 70, dtype: int64

year_sold value counts:
year_sold
2014    14633
2015     6980
Name: count, dtype: int64

month_sold value counts:
month_sold
5     2414
4  

## Conclusiones 
El dataset contiene información sobre 21 características de viviendas vendidas en una región, con un total de 21,613 registros. Las variables incluyen datos numéricos (como precio, habitaciones, baños, áreas, año de construcción) y categóricos (como vista al agua, calidad de la vista, condición, calificación de la construcción y código postal).

Para el análisis, se realizó la siguiente manipulación de los datos:

Se extrajeron el año y el mes de venta desde la columna date para facilitar análisis temporales.
Se creó la variable years_since_renovation para identificar el tiempo desde la última renovación o construcción.
Se separaron las variables en dos grupos: categóricas (por ejemplo, waterfront, view, condition, grade, zipcode, year_sold, month_sold) y no categóricas (por ejemplo, price, bedrooms, bathrooms, sqft_living, etc.).
Se revisaron los datos en busca de valores nulos y duplicados, confirmando la calidad del dataset.
Se realizó un resumen estadístico de las variables numéricas y un conteo de valores para las categóricas.
Esta manipulación permite entender la estructura del dataset y preparar los datos para el análisis exploratorio, el preprocesamiento y la modelización.